In [1]:
"""
This notebook is used to test the langchain library. We are looking to test the index / retrieval capabilities of the library.
langchain is a python library that allows you interact with LLM (large language model) models. 

We explore here the interaction with Juridicial corpus.

"""
import langchain

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQAWithSourcesChain
from langchain import OpenAI

import os
import pandas as pd

# We load the OpenAI API key from the environment variable OPENAI_API_KEY
# we first read the key from the key.txt file (not in the repo)
with open("../key.txt", "r") as f:
    key = f.read()

os.environ["OPENAI_API_KEY"] = key



In [12]:
df_codes_civil = pd.read_parquet("../data/df_law_civique.parquet")

# replace the nan values with empty strings
df_codes_civil = df_codes_civil.fillna("none")

texts = df_codes_civil["text"].tolist()
articles = df_codes_civil["article_number"].tolist()

embeddings = OpenAIEmbeddings()


In [13]:
len(texts)

2795

In [14]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": article_name} for article_name in articles])

Using embedded DuckDB without persistence: data will be transient


In [15]:
chain = RetrievalQAWithSourcesChain.from_chain_type(OpenAI(temperature=0), chain_type="stuff", retriever=docsearch.as_retriever())

In [17]:
chain({"question": "Que faire quand un locataire rend un bien anciennement loué dans un état dégradé ?"}, return_only_outputs=False)


{'question': 'Que faire quand un locataire rend un bien anciennement loué dans un état dégradé ?',
 'answer': " Le locataire doit rendre la chose telle qu'il l'a reçue, suivant l'état des lieux, excepté ce qui a péri ou a été dégradé par vétusté ou force majeure.\n",
 'sources': '1730, 1755, 1631'}

In [56]:
texts[1620]

'à la charge du bailleur : Travaux en cas de vétusté ou force majeure\nCirculaires et Instructions\n> CIRCULAIRE N° 90000/GEND/DSF/SDIL relative à l\'établissement des états des lieux des logements d\'habitation détenus par la gendarmerie.\n1756     \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nLe curement des puits et celui des fosses d\'aisances sont à la charge du bailleur s\'il n\'y a clause contraire.\nservice-public.fr\n> Entretien courant et "réparations locatives" à la charge du locataire : Fosse septique\n1757     \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nLe bail des meubles fournis pour garnir une maison entière, un corps de logis entier, une boutique, ou tous\nautres appartements, est censé fait pour la durée ordinaire des baux de maison, corps de logis, boutiques ou\nautres appartements, selon l\'usage des lieux.\n1758     \n Legif.   \n Plan   \n Jp.C.Cass.   \n Jp.Appel   \n Jp.Admin.   \n Juricaf\nLe b